In [ ]:
import numpy as np
import sys
sys.path.append("/Users/utilizador/Documents/GitHub/si/src")
from si.model_selection.cross_validate import k_fold_cross_validation
from si.io.csv_file import read_csv
from si.data.dataset import Dataset
from si.models.logistic_regression import LogisticRegression
from si.model_selection.split import train_test_split
from typing import Callable, Tuple, Dict, Any

A função randomized_search_cv implementa uma técnica de otimização de hiperparâmetros baseada em busca aleatória. Ela testa combinações aleatórias de valores de hiperparâmetros fornecidos por meio de validação cruzada para encontrar uma configuração que maximize (ou minimize) uma métrica de desempenho.

__Otimização de Hiperparâmetros:__
Hiperparâmetros são parâmetros de modelos que não são aprendidos durante o treinamento. Exemplos incluem a taxa de aprendizado em redes neurais ou o número de árvores em um Random Forest.
A escolha correta de hiperparâmetros pode melhorar significativamente o desempenho do modelo.

__Validação Cruzada (Cross-Validation):__
Divide os dados em múltiplos subconjuntos (folds). Em cada iteração, um fold é usado como conjunto de teste enquanto os demais são usados para treinamento.
A média das pontuações nos folds é usada como métrica de desempenho.

__Busca Aleatória (Randomized Search):__
Em vez de testar exaustivamente todas as combinações de valores (como na busca em grid), a busca aleatória seleciona combinações aleatórias. Isso reduz o custo computacional.

__Componentes:__

__model:__ O modelo cujo desempenho será avaliado.

__dataset:__ O conjunto de dados usado para a validação cruzada.

__hyperparameter_grid:__ Dicionário contendo os hiperparâmetros e seus valores possíveis.

__scoring:__ Função de avaliação (como acurácia ou erro quadrático médio).

__cv:__ Número de folds na validação cruzada.

__n_iter:__ Número de combinações aleatórias de hiperparâmetros a testar.

__Validação de Entradas:__
Verifica se o número de iterações, o grid de hiperparâmetros e o número de folds são válidos.
Certifica-se de que o modelo possui os hiperparâmetros fornecidos.

__Iterações Aleatórias:__
Para cada iteração, escolhe uma combinação aleatória de valores de hiperparâmetros.
Ajusta os hiperparâmetros no modelo e realiza validação cruzada.

__Armazenamento de Resultados:__
Armazena as pontuações e as combinações de hiperparâmetros testadas.

__Seleção do Melhor Conjunto:__
Identifica os hiperparâmetros e a pontuação que obtiveram o melhor desempenho.

In [1]:
def randomized_search_cv( model, dataset, hyperparameter_grid: Dict[str, Tuple], scoring: Callable = None, cv: int = 5,
    n_iter: int = None,) :
    
    """
    Performs random search of hyperparameters with cross-validation.

    Parameters:
    ----------
    model : Model
        The model to be evaluated.
    dataset : Dataset
        The dataset for cross-validation.
    hyperparameter_grid : Dict[str, Tuple]
        Dictionary with hyperparameters and their possible values.
    scoring : Callable
        Evaluation function for the model.
    cv : int
        Number of folds for cross-validation.
    n_iter : int
        Number of random combinations of hyperparameters to be tested.

    Returns:
    -------
    results : Dict[str, Any]
        Results of the random search, including scores and the best hyperparameters.
"""

    # Validação dos parâmetros
    if n_iter is None or n_iter <= 0:
        raise ValueError("O número de iterações (n_iter) deve ser maior que zero.")
    if not hyperparameter_grid or not isinstance(hyperparameter_grid, dict):
        raise ValueError("O grid de hiperparâmetros deve ser um dicionário válido.")
    if cv <= 1:
        raise ValueError("O número de folds (cv) deve ser maior que 1.")

    # Verifica se os hiperparâmetros existem no modelo
    for parameter in hyperparameter_grid:
        if not hasattr(model, parameter):
            raise AttributeError(f"Modelo {model} não possui o hiperparâmetro '{parameter}'.")

    results = {'scores': [], 'hyperparameters': []}

    for _ in range(n_iter):
        # Seleção aleatória de valores para os hiperparâmetros
        parameters = {key: np.random.choice(values) for key, values in hyperparameter_grid.items()}
        for key, value in parameters.items():
            setattr(model, key, value)  # Define o hiperparâmetro no modelo

        # Validação cruzada
        score = k_fold_cross_validation(model=model, dataset=dataset, scoring=scoring, cv=cv)

        # Armazena os resultados
        results['scores'].append(np.mean(score))
        results['hyperparameters'].append(parameters)

    # Identifica os melhores hiperparâmetros
    best_index = np.argmax(results['scores'])
    results['best_hyperparameters'] = results['hyperparameters'][best_index]
    results['best_score'] = results['scores'][best_index]

    return results




In [2]:
Path= "/Users/utilizador/Documents/GitHub/si/datasets/breast_bin/"
data = read_csv(Path + "breast-bin.csv", sep=",", label=True)
data.summary()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8
mean,4.412607,3.133238,3.206304,2.809456,3.217765,3.478510,3.438395,2.866762,1.590258
median,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
var,7.909984,9.310328,8.831364,8.148507,4.901002,13.074753,5.945345,9.324655,2.940994


In [3]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [4]:
model = LogisticRegression()

# Define hyperparameter distributions
l2_penalty = np.linspace(1, 10, 10)
alpha = np.linspace(0.001, 0.0001, 100)
max_iter = np.linspace(1000, 2000, 200, dtype=int)

parameter_grid_ = {
        'l2_penalty': (1, 10),
        'alpha': (0.001, 0.0001),
        'max_iter': (1000, 2000)
    }


# cross validate the model
results = randomized_search_cv(model=model,
                              dataset=train_data,
                              hyperparameter_grid=parameter_grid_,
                              cv=3,
                              n_iter=8)



In [5]:
# Output results
print("Randomized Search Results:")
print(f"Hyperparameters tested: {results['hyperparameters']}")
print(f"Scores: {results['scores']}")
print(f"Best Hyperparameters: {results['best_hyperparameters']}")
print(f"Best Score: {results['best_score']}")

Randomized Search Results:
Hyperparameters tested: [{'l2_penalty': np.int64(1), 'alpha': np.float64(0.001), 'max_iter': np.int64(1000)}, {'l2_penalty': np.int64(10), 'alpha': np.float64(0.001), 'max_iter': np.int64(1000)}, {'l2_penalty': np.int64(1), 'alpha': np.float64(0.0001), 'max_iter': np.int64(1000)}, {'l2_penalty': np.int64(10), 'alpha': np.float64(0.0001), 'max_iter': np.int64(2000)}, {'l2_penalty': np.int64(1), 'alpha': np.float64(0.0001), 'max_iter': np.int64(2000)}, {'l2_penalty': np.int64(1), 'alpha': np.float64(0.0001), 'max_iter': np.int64(1000)}, {'l2_penalty': np.int64(10), 'alpha': np.float64(0.0001), 'max_iter': np.int64(1000)}, {'l2_penalty': np.int64(10), 'alpha': np.float64(0.001), 'max_iter': np.int64(1000)}]
Scores: [np.float64(0.9611451942740287), np.float64(0.9611451942740287), np.float64(0.9611451942740286), np.float64(0.9611451942740287), np.float64(0.9611451942740287), np.float64(0.9591002044989775), np.float64(0.9611451942740287), np.float64(0.9611451942740